Aqui comenzare con las funciones y el sistema de recomendacion.

Importamos las librerias

In [1]:
import pandas as pd
import pyarrow.parquet as pq

Cargamos los archivos

In [2]:
reviews = pd.read_csv('nueva data/reviews.csv')
items = pq.read_table("nueva data/items_parquet.parquet").to_pandas()
games = pd.read_csv("nueva data/games.csv")
games_model = pd.read_csv('nueva data/games_model.csv')

Primera funcion:

In [7]:
def developer(dev,games,items):
    #Eliminamos las filas vacias en price y id
    games.dropna(subset=['price','id'],inplace=True)

    #Cambiamos los tipos de datos de games['id'] y items['id']
    games['id'] = games['id'].astype(int)
    items['id'] = items['id'].astype(int)

    #Filtramos games por el desarrolador seleccionado y reseteamos el indice
    games_developer_select = games[games['developer'] == dev.capitalize()].reset_index()

    #seleccionamos las columnas con las que trabajaremos
    games_dev_all = games_developer_select[['release_date','id']]

    #hacemos un primer merge para unir los juegos con los items
    merge = pd.merge(items[['id','user_id']],games_dev_all,on='id')

    #hacemos un conteo de items por año, reseteamos el indice y renombramos la columna
    recuento = merge.groupby('release_date').agg({'user_id':'count'}).reset_index()
    recuento.rename({'user_id':'cantidad_items'},axis=1,inplace=True)

    #Ahora filtramos los juegos gratis
    games_dev_free = games_developer_select[games_developer_select['price'] == '0']

    #ahora hacemos un merge con los juegos gratis y items, tambien hacemos un groupby con un recuento de items gratis y renombramos una columna
    merge_2 = pd.merge(games_dev_free[['release_date','id']],items[['id','user_id']],on='id')
    recuento_2 = merge_2.groupby('release_date').agg({'user_id':'count'}).reset_index()
    recuento_2.rename({'user_id':'cantidad_items_free'},axis=1,inplace=True)

    #creamos un tercer merge con los 2 recuentos
    merge_3 = recuento.merge(recuento_2,on='release_date',how='left').fillna(0)

    #calculamos y pasamos a porcentaje la columna calculada
    merge_3['contenido_Free'] = (recuento_2['cantidad_items_free'] / recuento['cantidad_items'] * 100).fillna(0)
    merge_3['contenido_Free'] = merge_3['contenido_Free'].apply(lambda x: f'{x}%')

    #por ultimo eliminamos la columna cantidad_items_free y renombramos release_date
    merge_3.drop('cantidad_items_free',axis=1,inplace=True)
    merge_3.rename({'release_date':'año'},axis=1,inplace=True)

    return merge_3

Realizamos una prueba: 

In [8]:
developer('valve',games,items)

,año,cantidad_items,contenido_Free
0,1998,7750,0.0%
1,1999,6334,0.0%
2,2000,13044,0.0%
3,2001,3274,0.0%
4,2003,3465,0.0%
5,2004,72060,0.0%
6,2006,26602,0.0%
7,2007,31834,0.0%
8,2008,6585,0.0%
9,2009,37044,0.0%


Segunda funcion:

In [5]:
def userdata(user,games,items,reviews):

    #realizamos un filtro con el usuario ingresado
    user = items[items['user_id'] == str(user)].reset_index(drop=True)

    #hacemos un merge para obtener datos de games y cambiamos el tipo de dato de la columna price
    merge = user.merge(games[['id','price']])
    merge['price'] = merge['price'].astype(float)

    #realizamos un groupby de el usuario, un recuento con el id, una suma de price y un reseteo del indice
    merge_1 = merge.groupby('user_id').agg({'id':'count','price':'sum'}).reset_index()

    #realizamos un merge de los datos del usuario filtrado con reviews y eliminamos duplicados
    user_reviews = user.merge(reviews[['user_id','recommend']])
    user_reviews.drop_duplicates(subset='id',inplace=True)

    #realizamos un groupby para realizar un recuento de reviews y reseteamos el indice
    Cantidad_reviews = user_reviews.groupby('user_id').agg({'id':'count'}).reset_index()

    #en 2 variables guardamos la cantidad de reviews que hizo el usuario y la cantidad de reviews recomendadas
    cantidad = Cantidad_reviews['id'][0]
    recomendados = user_reviews['recommend'].sum()

    #retornamos
    return (f"Usuario: {merge_1['user_id'][0]}, dinero gastado: {merge_1['price'][0]}$, porcentaje de recomendacion: {(recomendados//cantidad)*100}%, cantidad de items: {merge_1['id'][0]}")

Realizamos una prueba

In [6]:
userdata(76561197970982479,games,items,reviews)

'Usuario: 76561197970982479, dinero gastado: 3399.33$, porcentaje de recomendacion: 100%, cantidad de items: 226'

Tercera Funcion:

In [7]:
def UserForGenre(genre,games,items):
    #eliminamos los datos nulos en la columna genres
    games.dropna(subset='genres',inplace=True)

    #aplicamos un filtro con el genero ingresado
    juegos_genero_steam = games[games['genres'].str.lower().str.contains(genre)]
    juegos_genero_items = items[items['game'].isin(juegos_genero_steam['game'])]

    #hacemos un merge
    df_combinado = pd.merge(juegos_genero_items, juegos_genero_steam[['game', 'release_date']], on='game')

    #hacemos un groupby de los diferentes usuarios, hacemos una suma de sus horas jugadas y reseteamos el indice
    playtime_user = df_combinado.groupby('user_id').agg({'playtime_forever':'sum'}).reset_index()

    #obtenemos el usuario con mas horas jugadas de ese genero
    user = playtime_user[playtime_user['playtime_forever'] == (playtime_user['playtime_forever'].max())]
    user = user['user_id'].values[0]

    #filtramos ese usuario, hacemos un groupby con cada año y sus horas jugadas
    data_user = df_combinado[df_combinado['user_id'] == user]
    playtime_year = data_user.groupby('release_date').agg({'playtime_forever':'sum'}).reset_index()

    #hacemos una lista y un for para ir cargando los diccionarios a nuestra lista
    lista = []
    for i in range(len(playtime_year['release_date'].values)):
        dic = {'año':playtime_year['release_date'].values[i], 'horas':playtime_year['playtime_forever'].values[i]}
        lista.append(dic)

    #retornamos los resultados
    return (f"El usuario con mas horas jugadas para el genero:{genre} es:{user}, horas jugadas: {lista}")

Realizamos una prueba:

In [8]:
UserForGenre('action',games,items)

"El usuario con mas horas jugadas para el genero:action es:shinomegami, horas jugadas: [{'año': 1988, 'horas': 136.0}, {'año': 1991, 'horas': 0.0}, {'año': 1992, 'horas': 0.0}, {'año': 1993, 'horas': 0.0}, {'año': 1994, 'horas': 0.0}, {'año': 1995, 'horas': 2.0}, {'año': 1996, 'horas': 1161.0}, {'año': 1997, 'horas': 99.0}, {'año': 1998, 'horas': 3716.0}, {'año': 1999, 'horas': 303.0}, {'año': 2000, 'horas': 10775.0}, {'año': 2001, 'horas': 2429.0}, {'año': 2002, 'horas': 0.0}, {'año': 2003, 'horas': 532756.0}, {'año': 2004, 'horas': 120.0}, {'año': 2005, 'horas': 343.0}, {'año': 2006, 'horas': 2043.0}, {'año': 2007, 'horas': 68354.0}, {'año': 2008, 'horas': 180.0}, {'año': 2009, 'horas': 209805.0}, {'año': 2010, 'horas': 9405.0}, {'año': 2011, 'horas': 89382.0}, {'año': 2012, 'horas': 18284.0}, {'año': 2013, 'horas': 146583.0}, {'año': 2014, 'horas': 92080.0}, {'año': 2015, 'horas': 101449.0}, {'año': 2016, 'horas': 42563.0}, {'año': 2017, 'horas': 135.0}]"

Cuarta Funcion: 

In [9]:
def best_developer_year(year,games,reviews):
    #filtramos los datos de games por año
    games_year = games[games['release_date'] == int(year)]

    #hacemos un condicional para que si no hay datos vacios le devuelva un mensaje al usuario
    # y si hay datos guarde el merge en una variable
    if games_year.merge(reviews,on='id').empty:
        return ("No hay registros de este año, ingrese otro año")
    games_year_reviews = games_year.merge(reviews,on='id')

    #hacemo un filtro para obtener los recomendados y los de reviews positivos
    games_year_reviews[(games_year_reviews['recommend'] == 1) & (games_year_reviews['sentiment analysis'] == 2)]

    #hacemos una agrupacion de los desarrolladores y realizamos 
    dev_recommend = games_year_reviews.groupby('developer').agg({'recommend':'count','sentiment analysis':'count'}).reset_index()

    #realizamos una suma de los conteos y guardamos en una nueva columna
    dev_recommend['suma_recommend'] = dev_recommend['recommend'] + dev_recommend['sentiment analysis']

    #obtenemos los indices
    indices = dev_recommend['suma_recommend'].nlargest(3).index

    return(f"Puesto 1: {dev_recommend['developer'][indices[0]]}, Puesto 2: {dev_recommend['developer'][indices[1]]}, Puesto 3: {dev_recommend['developer'][indices[2]]}")

Realizamos una prueba:

In [10]:
best_developer_year(2017,games,reviews)

'Puesto 1: Smartly Dressed Games, Puesto 2: Freejam, Puesto 3: Studio Wildcard,Instinct Games,Efecto Studios,Virtual Basement LLC'

Quinta Funcion:

In [32]:
def developer(dev,games,reviews):
    #aplicamos un filtro para tener solo el desarollador ingresado
    games_dev = games[games['developer'] == str(dev).capitalize()]

    #hacemos una condicional para indicar si el desarrollador ingresado es valido o si hay datos
    if games_dev.empty:
        ("no se encuentran datos de el desarrolador ingresado, intente otro")

    #realizamos un merge, hacemos un groupby con el recuento de los analisys y reseteamos el indice
    merge = games_dev.merge(reviews,on='id')
    merge = merge.groupby('sentiment analysis').agg({'developer':'count'}).reset_index()
    #nos quedamos con las filas que no sean neutras 
    merge = merge[merge['sentiment analysis'] != 1]  
    #retornamos
    return {dev:list(merge['developer'].values)}

Realizamos una prueba:

In [33]:
developer('Valve',games,reviews)

{'Valve': [1219, 6149]}

Seguire con los sistemas de recomendacion:

Item-item:

In [82]:
def item_item(id,games_model):
    #hacemos un copia de nuestro dataframe games_model y elimninamos las columnas id y game
    games_copy = games_model.copy()
    games_copy.drop(['game','id'],axis=1,inplace=True)

    #importamos el cosine_similarity 
    from sklearn.metrics.pairwise import cosine_similarity

    #reemplazamos los datos de games_copy con los resultados de la funcion cosine_similarity y convertimos en un dataframe
    games_copy = cosine_similarity(games_copy)
    games_copy = pd.DataFrame(games_copy)

    #obtenemos el indice del id del juego ingresado
    index = games_model[games_model['id'] == id].index[0]

    #obtenemos los indices de las mejores recomendaciones
    recommend = games_copy[index].sort_values(ascending=False).iloc[1:6].index

    #localizamos las recomendaciones y obtenemos el id y el nombre
    recommend = games_model.loc[recommend, ['id','game']].values

    return(f'estos son los juegos recomendados con sus ids y nombres: 1 id:{int(recommend[0][0])}  name: {recommend[0][1]}, 2 id:{int(recommend[1][0])} name: {recommend[1][1]}, 3 id:{int(recommend[2][0])} name: {recommend[2][1]}, 4 id:{int(recommend[3][0])} name: {recommend[3][1]}, 5 id:{int(recommend[4][0])} name: {recommend[4][1]}.')

Realizamos un prueba:

In [83]:
item_item(10,games_model)

'estos son los juegos recomendados con sus ids y nombres: 1 id:60  name: Ricochet, 2 id:2350 name: QUAKE III: Team Arena, 3 id:560170 name: STAR WARS™ SHADOWS OF THE EMPIRE™, 4 id:32380 name: STAR WARS™ Jedi Knight: Dark Forces II, 5 id:356210 name: Tony Tough and the Night of Roasted Moths.'

User-items:

In [84]:
def user_item(user,items,games_model):
    # filtramos los usuarios para traer todos los juegos jugados por el.
    items = items[items['user_id'] == str(user)]
    #traeré el id del juego mas jugado por ese usuario.
    max = items['playtime_forever'].idxmax()
    id_game = items['id'].iloc[max]
    #Usaré el id del juego y lo metemos a la función que habia creado antes para encontrar los 5 juegos recomendados
    return item_item(id_game,games_model)

Realizamos un prueba:

In [85]:
user_item(76561197970982479,items,games_model)

'estos son los juegos recomendados con sus ids y nombres: 1 id:211994  name: Hacker Evolution Duality: Inception Part 2 DLC, 2 id:221020 name: Towns, 3 id:356550 name: East Tower - Kuon, 4 id:32160 name: Zombie Bowl-o-Rama, 5 id:389460 name: Robot vs Birds Zombies.'